In [3]:
from pyspark.sql import functions as F
input_bucket = 's3://zhan-commoncrawl/RQX/all_urls/'
# input_bucket = "s3://zhan-commoncrawl/DK_websites/dk_2021_websites/"
# df = spark.read.csv(input_bucket,header = False).toDF("crawl","url_host_name","url_host_registered_domain","warc_filename","url","warc_record_offset","warc_record_length"
# )
df = spark.read.parquet(input_bucket)

a = 'CC-MAIN-2015-32'
df.createOrReplaceTempView("education_file_list")
sqlDF = spark.sql('SELECT * from education_file_list where crawl="{}"'.format(a))
# sqlDF.show()
warc_filename_list = sqlDF.collect()[:10]

print(warc_filename_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(crawl='CC-MAIN-2015-32', url_host_name='www.justincase.be', url_host_registered_domain='justincase.be', warc_filename='crawl-data/CC-MAIN-2015-32/segments/1438044160065.87/warc/CC-MAIN-20150728004240-00006-ip-10-236-191-2.ec2.internal.warc.gz', url='http://www.justincase.be/', warc_record_offset=532355587, warc_record_length=4905, ranking=1), Row(crawl='CC-MAIN-2015-32', url_host_name='www.hotel-leopold.be', url_host_registered_domain='hotel-leopold.be', warc_filename='crawl-data/CC-MAIN-2015-32/segments/1438042987775.70/warc/CC-MAIN-20150728002307-00076-ip-10-236-191-2.ec2.internal.warc.gz', url='http://www.hotel-leopold.be/', warc_record_offset=504234759, warc_record_length=7503, ranking=1), Row(crawl='CC-MAIN-2015-32', url_host_name='www.thedeafnun.be', url_host_registered_domain='thedeafnun.be', warc_filename='crawl-data/CC-MAIN-2015-32/segments/1438042986022.41/warc/CC-MAIN-20150728002306-00301-ip-10-236-191-2.ec2.internal.warc.gz', url='http://www.thedeafnun.be/', warc_recor

## 获得trackers

In [4]:
tracker_bucket = 's3://aws-emr-resources-235671948910-us-east-1/labeled_third_party/'
df_tracker = spark.read.option("header",True).csv(tracker_bucket)
df_tracker.createOrReplaceTempView("tracker_list")
sqlDF_tracker = spark.sql("SELECT Domain, Category, Company from tracker_list")
sqlDF_tracker.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------+
|              Domain|            Category|    Company|
+--------------------+--------------------+-----------+
|google-analytics.com|           Analytics|     Google|
|googlesyndication...|         Advertising|     Google|
|          google.com|                   #|     Google|
|      googleapis.com|                   #|     Google|
|        facebook.com|         Advertising|   Facebook|
|      macromedia.com|                   #|      Adobe|
|         youtube.com|                   #|     Google|
|           adobe.com|                   #|      Adobe|
|        facebook.net|         Advertising|   Facebook|
|         twitter.com|         Advertising|    Twitter|
|     casalemedia.com|         Advertising|CasaleMedia|
|         addthis.com|              Widget|    AddThis|
|        gravatar.com|              Widget| Automattic|
|            gmpg.org|                   #|          #|
|          imgaft.com|                   #|     

In [5]:
tracker_bucket = 's3://aws-emr-resources-235671948910-us-east-1/edu_trackers/'
df_tracker = spark.read.option("header",True).csv(tracker_bucket)
df_tracker.createOrReplaceTempView("tracker_list")
sqlDF_tracker = spark.sql("SELECT domain from tracker_list")
print(sqlDF_tracker.show())
tracker_list = sqlDF_tracker.select('domain').rdd.flatMap(lambda x:x).collect()
print(tracker_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              domain|
+--------------------+
|  123contactform.com|
|             2o7.net|
|           4stats.de|
|               4u.pl|
|               51.la|
|              a8.net|
|     accesstrade.net|
|         adbrite.com|
|         addthis.com|
|        addtoany.com|
|           adjug.com|
|       admission.net|
|          adonweb.ru|
|          adriver.ru|
|          adroll.com|
|          adtaily.pl|
|           adtech.de|
|adultfriendfinder...|
|     advertising.com|
|    advertstream.com|
+--------------------+
only showing top 20 rows

None
['123contactform.com', '2o7.net', '4stats.de', '4u.pl', '51.la', 'a8.net', 'accesstrade.net', 'adbrite.com', 'addthis.com', 'addtoany.com', 'adjug.com', 'admission.net', 'adonweb.ru', 'adriver.ru', 'adroll.com', 'adtaily.pl', 'adtech.de', 'adultfriendfinder.com', 'advertising.com', 'advertstream.com', 'affili.net', 'affiliate-b.com', 'affiliatefuture.com', 'akavita.by', 'amazingcounters.com', 'amung.us', 'andyhopp

In [6]:
tracker_list = sqlDF_tracker.select('Domain').rdd.flatMap(lambda x:x).collect()
print(tracker_list[:10])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['123contactform.com', '2o7.net', '4stats.de', '4u.pl', '51.la', 'a8.net', 'accesstrade.net', 'adbrite.com', 'addthis.com', 'addtoany.com']

In [11]:
class Args:
    warc_parse_http_header = True
    records_processed = None
    warc_input_processed = None
    warc_input_failed = None
    num_input_partitions = 10
    num_output_partitions = 5
    output = "s3://zhan-commoncrawl/tag_count_output"
    output_format = "csv"
    output_compression = "gzip"
    output_option = []
    local_temp_dir = None
    spark_profiler = False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
import argparse
import logging
import os
import re
from urllib.parse import urlparse

from io import BytesIO
from tempfile import TemporaryFile

import boto3
import botocore

from warcio.archiveiterator import ArchiveIterator
from warcio.recordloader import ArchiveLoadFailed

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType
import tempfile

LOGGING_FORMAT = '%(asctime)s %(levelname)s %(name)s: %(message)s'

    
class JupyterCCSparkJob(object):
    """
    A simple Spark job definition to process Common Crawl data
    """

    name = 'CCSparkJob'

    output_schema = StructType([
        StructField("key", StringType(), True),
        StructField("val", StringType(), True)
    ])

    # description of input and output shown in --help
    input_descr = "Path to file listing input paths"
    output_descr = "Name of output table (saved in spark.sql.warehouse.dir)"

    warc_parse_http_header = True

    args = None
    records_processed = None
    warc_input_processed = None
    warc_input_failed = None
    log_level = 'INFO'
    logging.basicConfig(level=log_level, format=LOGGING_FORMAT)


    def parse_arguments(self):
        """ Returns the parsed arguments from the command line """

        description = self.name
        if self.__doc__ is not None:
            description += " - "
            description += self.__doc__
        args = Args()
        return args

    def add_arguments(self, parser):
        pass

    def validate_arguments(self, args):
        if "orc" == args.output_format and "gzip" == args.output_compression:
            # gzip for Parquet, zlib for ORC
            args.output_compression = "zlib"
        return True

    def get_output_options(self):
        return {x[0]: x[1] for x in map(lambda x: x.split('=', 1),
                                        self.args.output_option)}

    def init_logging(self, level=None):
        if level is None:
            level = self.log_level
        else:
            self.log_level = level
        logging.basicConfig(level=level, format=LOGGING_FORMAT)

    def init_accumulators(self, sc):
        self.records_processed = sc.accumulator(0)
        self.warc_input_processed = sc.accumulator(0)
        self.warc_input_failed = sc.accumulator(0)

    def get_logger(self, spark_context=None):
        """Get logger from SparkContext or (if None) from logging module"""
        if spark_context is None:
            return logging.getLogger(self.name)
        return spark_context._jvm.org.apache.log4j.LogManager \
            .getLogger(self.name)

    def run(self):
        self.args = self.parse_arguments()
        conf = SparkConf()
        conf.set("spark.hadoop.validateOutputSpecs", "false")

        
        
        if self.args.spark_profiler:
            conf = conf.set("spark.python.profile", "true")
        
        sc = SparkContext.getOrCreate(conf=conf)
        
        
        sqlc = SQLContext(sparkContext=sc)

        self.init_accumulators(sc)

        self.run_job(sc, sqlc)

        if self.args.spark_profiler:
            sc.show_profiles()

#         sc.stop()

    def log_aggregator(self, sc, agg, descr):
        self.get_logger(sc).info(descr.format(agg.value))

    def log_aggregators(self, sc):
        self.log_aggregator(sc, self.warc_input_processed,
                            'WARC/WAT/WET input files processed = {}')
        self.log_aggregator(sc, self.warc_input_failed,
                            'WARC/WAT/WET input files failed = {}')
        self.log_aggregator(sc, self.records_processed,
                            'WARC/WAT/WET records processed = {}')

    @staticmethod
    def reduce_by_key_func(a, b):
        return a + b
    
    def to_file(self, data):
        url = str(data[0])
        trackres = ','.join(data[1])
        return '\t'.join([url,trackers])

    def run_job(self, sc, sqlc):
        
        input_data = sc.parallelize(warc_filename_list)
        print(input_data)
        output = input_data.mapPartitionsWithIndex(self.process_warcs)\
        .reduceByKey(lambda x,y:x)
        print(output.count())
        print(output.take(100))
        columns = ['url','trackers']
        df = output.toDF(columns)
        df.write.format("csv").mode('overwrite').option("header", "true").save(self.args.output)
        
        self.log_aggregators(sc)
    
    def process_warcs(self, id_, iterator):
        s3pattern = re.compile('^s3://([^/]+)/(.+)')
        base_dir = "/user/"

        # S3 client (not thread-safe, initialize outside parallelized loop)
        no_sign_request = botocore.client.Config(
            signature_version=botocore.UNSIGNED)
        s3client = boto3.client('s3', config=no_sign_request)
       
        for item in iterator:
            warc_filename = item['warc_filename']
            offset = item['warc_record_offset']
            length = item['warc_record_length']
            
            crawl_time = item['crawl']
            self.warc_input_processed.add(1)
           
            self.get_logger().info('Reading from S3 {}'.format(warc_filename))
            
            offset_end = int(offset) + int(length) - 1
            byte_range = 'bytes={offset}-{end}'.format(offset=offset, end=offset_end)

            warctemp = None
            try:
                warctemp = s3client.get_object(Bucket='commoncrawl', Key=warc_filename, Range=byte_range)['Body']
            except botocore.client.ClientError as exception:
                self.get_logger().error(
                    'Failed to access {}: {}'.format(warc_filename, exception))
                self.warc_input_failed.add(1)
                continue
            stream = warctemp

            no_parse = (not self.warc_parse_http_header)
           
            try:
                archive_iterator = ArchiveIterator(stream,
                                                   no_record_parse=no_parse, arc2warc = True)
                
                for res in self.iterate_records(crawl_time, archive_iterator):

                    yield res
            except ArchiveLoadFailed as exception:
                self.warc_input_failed.add(1)
                self.get_logger().error(
                    'Invalid WARC: {} - {}'.format(warc_filename, exception))
            finally:
                stream.close()

    def process_record(self,record,crawl_time = None):
        raise NotImplementedError('Processing record needs to be customized')

    def iterate_records(self, crawl_time, archive_iterator):
        """Iterate over all WARC records. This method can be customized
           and allows to access also values from ArchiveIterator, namely
           WARC record offset and length."""
    
        for record in archive_iterator:
            rec_type = record.rec_type
            url = record.rec_headers.get_header('WARC-Target-URI')
            for res in self.process_record(record,crawl_time):   
                yield res
         
            self.records_processed.add(1)
            # WARC record offset and length should be read after the record
            # has been processed, otherwise the record content is consumed
            # while offset and length are determined:
            #  warc_record_offset = archive_iterator.get_record_offset()
            #  warc_record_length = archive_iterator.get_record_length()

    @staticmethod
    def is_wet_text_record(record):
        """Return true if WARC record is a WET text/plain record"""
        return (record.rec_type == 'conversion' and
                record.content_type == 'text/plain')

    @staticmethod
    def is_wat_json_record(record):
        """Return true if WARC record is a WAT record"""
        return (record.rec_type == 'metadata' and
                record.content_type == 'application/json')

    @staticmethod
    def is_html(record):
        """Return true if (detected) MIME type of a record is HTML"""
        html_types = ['text/html', 'application/xhtml+xml']
        if (('WARC-Identified-Payload-Type' in record.rec_headers) and
            (record.rec_headers['WARC-Identified-Payload-Type'] in
             html_types)):
            return True
        for html_type in html_types:
            if html_type in record.content_type:
                return True
        return False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from bs4 import BeautifulSoup
from selectolax.parser import HTMLParser
import re
regex = "((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"

def get_text_bs(html):
    
    try:
        tree = BeautifulSoup(html, 'lxml')
    
        trackers = []

        body = tree.body
        if body is None:
            return None

        for tag in body.select('style'):
            tag.decompose()

        text = body.get_text(separator='\n')
        
        s = body.find_all('script')
        for ss in s:
            url = ss.get('src')
            domain = str(urlparse(url).netloc)
            domain = '.'.join(domain.split('.')[-2:])
            if len(domain) >= 2 and domain in tracker_list:
                trackers.append(domain)
                
        
        a = body.find_all('a')
        for aa in a:
            url = aa.get('href')
            domain = str(urlparse(url).netloc)

            domain = '.'.join(domain.split('.')[-2:])
            if len(domain) >= 2 and domain in tracker_list:
                trackers.append(domain)

      

    except Exception as e:
        print(e)
    
    return ','.join(list(set(trackers)))

def get_text_selectolax(html):
    trackers = []
    try:
        tree = HTMLParser(html)
        if tree.body is None:
            return None
        
        for node in tree.tags('style'):
            node.decompose()
            
        #         找到a
        for node in tree.css('a,link,script,iframe,img'):
            text = node.text()
            if ("google-analytics" in text):
                trackers.append("google-analytics.com")
            if 'href' in node.attributes:
                url = node.attributes['href']
                domain = str(urlparse(url).netloc)
#                 domain = '.'.join(domain.split('.')[-2:])
                if len(domain) >= 2:
                    trackers.append(domain) # 这里我们认为所有的third-party request 都是trackers
            if 'src' in node.attributes:             
                url = node.attributes['src']
                domain = str(urlparse(url).netloc)
#                 domain = '.'.join(domain.split('.')[-2:])
                if len(domain) >= 2:
                    trackers.append(domain)
                    
            if "type" in node.attributes and node.attributes['type'] == 'text/javascript':
                
                result = re.findall(regex,text)
                for url in result:
                    domain = str(urlparse(url).netloc)
#                     domain = '.'.join(domain.split('.')[-2:])
                    if len(domain) >= 2:
                        trackers.append(domain)
                        
    except Exception as e:
        print(e)
    
    return ','.join(list(set(trackers)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:

                    
class Trackers_extraction_job(JupyterCCSparkJob):
    """ Count HTML tag names in Common Crawl WARC files"""

    name = "TagCount"

    # match HTML tags (element names) on binary HTML data
    html_tag_pattern = re.compile(b'<([a-z0-9]+)')

    def process_record(self, record, crawl_time):
        
        if record.rec_type != 'response':
            return
        url = record.rec_headers.get_header('WARC-Target-URI')
        domain = urlparse(url).netloc
        text = record.content_stream().read()
        trackers = get_text_selectolax(text)
        if url and url.strip() != '': 
            yield "#".join([url,crawl_time]), trackers


            
job = Trackers_extraction_job()
job.run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
RDD is empty
Traceback (most recent call last):
  File "<stdin>", line 109, in run
  File "<stdin>", line 145, in run_job
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 58, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 746, in createDataFrame
    rdd, schema = self._createFromRDD(data.map(prepare), schema, samplingRatio)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 390, in _createFromRDD
    struct = self._inferSchema(rdd, samplingRatio, names=schema)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 361, in _inferSchema
    first = rdd.first()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 1381, in first
    raise ValueError("RDD is empty")
ValueError: RDD is empty

